# Using MCP Tools with SmolAgents

https://hungvtm.medium.com/building-mcp-servers-and-client-with-smolagents-bd9db2d640e6

## Define the MCP servers

In [3]:
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters
from smolagents import LiteLLMModel
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

model = LiteLLMModel(
    model_id="ollama_chat/qwen3:30b",
    api_base="http://localhost:11434", 
    api_key="ollama",
    num_ctx=32000,
)

model = LiteLLMModel(model_id="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# # Define MCP server parameters
# weather_server_parameters = StdioServerParameters(
#     command="uv",
#     args=["run", "python", "weather.py"],
#     env={"UV_PYTHON": "3.12", **os.environ},
# )
# pubme_server_parameters = StdioServerParameters(
#     command="uvx",
#     args=["--quiet", "pubmedmcp@0.1.3"],
#     env={"UV_PYTHON": "3.12", **os.environ},
# )

pubme_server_parameters = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"]
)

brave_search_mcp_server = StdioServerParameters(
    command= "npx",
    args=[
          "-y",
          "@modelcontextprotocol/server-brave-search"
        ],
    env= {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
)

tavily_search_mcp_server = StdioServerParameters(
    command= "npx",
    args=[
        "-y",
        "tavily-mcp@0.1.3"
        ],
    env= {"TAVILY_API_KEY": os.getenv("TAVILY_API_KEY")}
)

arxiv_mcp_server = StdioServerParameters(
    command= "uv",
    args=[
        "tool",
        "run",
        "arxiv-mcp-server",
        "--storage-path", "/Users/inigo/Documents/papers"
        ]
)

yfinance_mcp_server = StdioServerParameters(
    command= "uvx",
    args=[
        "yfmcp"
        ]
)


## Load tools from MCP servers

### Using brave MCP tool

In [3]:
# Load tools from MCP servers
with ToolCollection.from_mcp(brave_search_mcp_server, trust_remote_code=True) as brave_search_tool_collection:
   all_tools = [*brave_search_tool_collection.tools]
   # Create an agent with all tools 
   agent = CodeAgent(tools=all_tools, model=model, add_base_tools=True)
   # Visualize agent behavior
   agent.visualize()
   print(agent.run("What was the Spanish outage event?"))

CodeAgent | openai/gpt-4o
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name               ┃ Description                               ┃ Arguments                                  ┃
    ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ brave_web_search   │ Performs a web search using the Brave     │ query (`string`): Search query (max 400    │
    │                    │ Search API, ideal for general queries,    │ chars, 50 words)                           │
    │                    │ news, articles, and online content. Use   │ count (`number`): Number of results (1-20, │
    │                    │ this for broad information gathering,     │ default 10)                                │
    │                    │ recent events, or when you need diverse   │ offset (`number`): Pagination offset (max  │
    │                    │ web sources. Supports pagination, content │ 9, default 0)                              │
    │                    │ filtering, and freshness controls.        │                                            │
    │                    │ Maximum 20 results per request, with      │                                            │
    │                    │ offset for pagination.                    │                                            │
    │ brave_local_search │ Searches for local businesses and places  │ query (`string`): Local search query (e.g. │
    │                    │ using Brave's Local Search API. Best for  │ 'pizza near Central Park')                 │
    │                    │ queries related to physical locations,    │ count (`number`): Number of results (1-20, │
    │                    │ businesses, restaurants, services, etc.   │ default 5)                                 │
    │                    │ Returns detailed information including:   │                                            │
    │                    │ - Business names and addresses            │                                            │
    │                    │ - Ratings and review counts               │                                            │
    │                    │ - Phone numbers and opening hours         │                                            │
    │                    │ Use this when the query implies 'near me' │                                            │
    │                    │ or mentions specific locations.           │                                            │
    │                    │ Automatically falls back to web search if │                                            │
    │                    │ no local results are found.               │                                            │
    │ web_search         │ Performs a duckduckgo web search based on │ query (`string`): The search query to      │
    │                    │ your query (think a Google search) then   │ perform.                                   │
    │                    │ returns the top search results.           │                                            │
    │ visit_webpage      │ Visits a webpage at the given url and     │ url (`string`): The url of the webpage to  │
    │                    │ reads its content as a markdown string.   │ visit.                                     │
    │                    │ Use this to browse webpages.              │                                            │
    │ final_answer       │ Provides a final answer to the given      │ answer (`any`): The final answer to the    │
    │                    │ problem.                                  │ problem                                    │
    └────────────────────┴───────────────────────────────────────────┴────────────────────────────────────────────┘

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What was the Spanish outage event?                                                                              │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To find information on the "Spanish outage event," I will perform a web search to gather the required 
details. This will help in understanding what the event was about.

Let's begin by searching the web for this event.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 1.60 seconds| Input tokens: 2,310 | Output tokens: 44]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The Spanish outage event refers to a significant power outage that affected Spain and Portugal,    
  creating widespread disruption. The blackout impacted key infrastructures like subway networks, phone lines,     
  and traffic systems, affecting millions. Although the exact cause was initially unclear, power was eventually    
  restored. The event was described as 'exceptional and extraordinary.'")                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The Spanish outage event refers to a significant power outage that affected Spain and Portugal,
creating widespread disruption. The blackout impacted key infrastructures like subway networks, phone lines, and 
traffic systems, affecting millions. Although the exact cause was initially unclear, power was eventually restored.
The event was described as 'exceptional and extraordinary.'

[Step 3: Duration 4.97 seconds| Input tokens: 8,408 | Output tokens: 326]

The Spanish outage event refers to a significant power outage that affected Spain and Portugal, creating widespread disruption. The blackout impacted key infrastructures like subway networks, phone lines, and traffic systems, affecting millions. Although the exact cause was initially unclear, power was eventually restored. The event was described as 'exceptional and extraordinary.'


In [5]:
# Load tools from MCP servers
with ToolCollection.from_mcp(yfinance_mcp_server, trust_remote_code=True) as yfinance_tool_collection:
   all_tools = [*yfinance_tool_collection.tools]
   # Create an agent with all tools 
   agent = CodeAgent(tools=all_tools, model=model, add_base_tools=True)
   # Visualize agent behavior
   agent.visualize()
   print(agent.run("What is the short-term trend for NVDA stock?"))

CodeAgent | openai/gpt-4o
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name              ┃ Description                                ┃ Arguments                                  ┃
    ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ get_ticker_info   │ Retrieve stock data including company      │ symbol (`string`): The stock symbol        │
    │                   │ info, financials, trading metrics and      │                                            │
    │                   │ governance data.                           │                                            │
    │ get_ticker_news   │ Fetches recent news articles related to a  │ symbol (`string`): The stock symbol        │
    │                   │ specific stock symbol with title, content, │                                            │
    │                   │ and source details.                        │                                            │
    │ search            │ Fetches and organizes search results from  │ query (`string`): The search query (ticker │
    │                   │ Yahoo Finance, including stock quotes and  │ symbol or company name)                    │
    │                   │ news articles.                             │ search_type (`string`): Type of search     │
    │                   │                                            │ results to retrieve                        │
    │ get_top           │ Get top entities (ETFs, mutual funds,      │ sector (`string`): The sector to get       │
    │                   │ companies, growth companies, or performing │ top_type (`string`): Type of top companies │
    │                   │ companies) in a sector.                    │ to retrieve                                │
    │                   │                                            │ top_n (`integer`): Number of top entities  │
    │                   │                                            │ to retrieve (limit the results)            │
    │ get_price_history │ Fetch historical price data for a given    │ symbol (`string`): The stock symbol        │
    │                   │ stock symbol over a specified period and   │ period (`string`): Time period to retrieve │
    │                   │ interval.                                  │ data for (e.g. '1d', '1mo', '1y')          │
    │                   │                                            │ interval (`string`): Data interval         │
    │                   │                                            │ frequency (e.g. '1d', '1h', '1m')          │
    │ web_search        │ Performs a duckduckgo web search based on  │ query (`string`): The search query to      │
    │                   │ your query (think a Google search) then    │ perform.                                   │
    │                   │ returns the top search results.            │                                            │
    │ visit_webpage     │ Visits a webpage at the given url and      │ url (`string`): The url of the webpage to  │
    │                   │ reads its content as a markdown string.    │ visit.                                     │
    │                   │ Use this to browse webpages.               │                                            │
    │ final_answer      │ Provides a final answer to the given       │ answer (`any`): The final answer to the    │
    │                   │ problem.                                   │ problem                                    │
    └───────────────────┴────────────────────────────────────────────┴────────────────────────────────────────────┘

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the short-term trend for NVDA stock?                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To determine the short-term trend for NVDA (NVIDIA Corporation) stock, I will follow these steps:

1. Retrieve the recent price history of NVDA stock to analyze its trend by using the `get_price_history` tool.
2. Analyze the retrieved price data to determine the trend.

First, let's fetch the historical price data for NVDA over the last month with daily intervals. 

Let's proceed with step 1.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 2.28 seconds| Input tokens: 2,375 | Output tokens: 87]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The short-term trend for NVDA stock is upward.")                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The short-term trend for NVDA stock is upward.

[Step 4: Duration 1.55 seconds| Input tokens: 13,766 | Output tokens: 456]

The short-term trend for NVDA stock is upward.


### Using pubmed MCP tool

In [5]:
# Load tools from MCP servers
with ToolCollection.from_mcp(pubme_server_parameters, trust_remote_code=True) as pubme_tool_collection:
   all_tools = [*pubme_tool_collection.tools]
   # Create an agent with all tools 
   agent = CodeAgent(tools=all_tools, model=model, add_base_tools=True)
   # Visualize agent behavior
   agent.visualize()
   print(agent.run("List the 3 most recent articles on 'Marfan syndrome', get the title, abstract, and authors and return them in a markdown table format."))

CodeAgent | openai/gpt-4o
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name             ┃ Description                                 ┃ Arguments                                  ┃
    ┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ search_abstracts │ Search abstracts on PubMed database based   │ request (`object`): Request parameters for │
    │                  │ on the request parameters.                  │ NCBI ESearch API for searching abstracts   │
    │                  │                                             │ on the PubMed database.                    │
    │                  │     While it returns a free-form text in    │                                            │
    │                  │ practice this is a list of strings          │ Functions:                                 │
    │                  │ containing:                                 │     - Provides a list of abstracts         │
    │                  │                                             │ matching a text query                      │
    │                  │     * the title of the article              │                                            │
    │                  │     * the abstract content                  │ Examples:                                  │
    │                  │     * the authors                           │     >>> # Basic search in PubMed for       │
    │                  │     * the journal name                      │ 'asthma' articles abstracts                │
    │                  │     * the publication date                  │     >>>                                    │
    │                  │     * the DOI                               │ SearchAbstractsRequest(term="asthma")      │
    │                  │     * the PMID                              │                                            │
    │                  │                                             │     >>> # Search with publication date     │
    │                  │     Args:                                   │ range                                      │
    │                  │         request: SearchAbstractsRequest     │     >>> ESearchRequest(                    │
    │                  │                                             │     ...     term="asthma",                 │
    │                  │                                             │     ...     mindate="2020/01/01",          │
    │                  │                                             │     ...     maxdate="2020/12/31",          │
    │                  │                                             │     ...     datetype="pdat"                │
    │                  │                                             │     ... )                                  │
    │                  │                                             │                                            │
    │                  │                                             │     >>> # Search with field restriction    │
    │                  │                                             │     >>> ESearchRequest(term="asthma")      │
    │                  │                                             │     >>> # Or equivalently:                 │
    │                  │                                             │     >>> ESearchRequest(term="asthma",      │
    │                  │                                             │ field="title")                             │
    │                  │                                             │                                            │
    │                  │                                             │     >>> # Search with proximity operator   │
    │                  │                                             │     >>> ESearchRequest(term='"as

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ List the 3 most recent articles on 'Marfan syndrome', get the title, abstract, and authors and return them in a │
│ markdown table format.                                                                                          │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for the most recent articles on Marfan syndrome                                                         
  request = search_abstracts(EXSearchRequest(term="Marfan syndrome", sort="pub_date"))                             
  print(request)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'request = search_abstracts(EXSearchRequest(term="Marfan syndrome", 
sort="pub_date"))' due to: InterpreterError: Forbidden function evaluation: 'EXSearchRequest' is not among the 
explicitly allowed tools or defined/imported in the preceding code

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Articles data from the observation                                                                             
  articles = [                                                                                                     
      {                                                                                                            
          "title": "Proximal stenting to resolve malperfusion before arch repair of a Marfan Type A dissection.",  
          "abstract": "Malperfusion has been reported as one of the strongest predictors of morbidity ...",        
          "authors": "Potti C, Henn M, Orion K, Neschis M, Bozinovski J, Tillman BW",                              
      },                                                                                                           
      {                                                                                                            
          "title": "Epigenome editing-mediated restoration of FBN1 expression by demethylation of CpG island       
  shore in porcine fibroblasts.",                                                                                  
          "abstract": "Fibrillin-1, an extracellular matrix protein encoded by the FBN1 gene, is crucial for       
  maintaining ...",                                                                                                
          "authors": "Miyadai R, Hinata S, Amemiya Y, Shigematsu S, Umeyama K, Nagashima H, Yamatoya K, Ohgane     
  J",                                                                                                              
      },                                                                                                           
      {                                                                                                            
          "title": "Endothelial Cell Senescence in Marfan Syndrome: Pathogenesis and Therapeutic Potential of      
  TGF-β Pathway Inhibition.",                                                                                      
          "abstract": "Marfan syndrome (MFS) is a heritable connective tissue disorder caused by mutations ...",   
          "authors": "Chen Y, Zhu Y, Ren X, Ding L, Xu Y, Zhou M, Dong R, Jin P, Chen X, Fan X, Li M, Gong Y,      
  Wang Y",                                                                                                         
      }                                                                                                            
  ]                                                                                                                
                                                                                                                   
  # Create markdown table                                                                                          
  markdown_table = "| Title | Abstract | Authors |\n"                                                              
  markdown_table += "|-------|----------|---------|\n"                                                             
                                                                                                                   
  for article in articles:                                                                                         
      markdown_table += f"| {article['title']} | {article['abstract']} | {article['authors']} |\n"                 
                                                                                                                   
  final_answer(markdown_table)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: | Title | Abstract | Authors |
|-------|----------|---------|
| Proximal stenting to resolve malperfusion before arch repair of a Marfan Type A dissection. | Malperfusion has 
been reported as one of the strongest predictors of morbidity ... | Potti C, Henn M, Orion K, Neschis M, Bozinovski
J, Tillman BW |
| Epigenome editing-mediated restoration of FBN1 expression by demethylation of CpG island shore in porcine 
fibroblasts. | Fibrillin-1, an extracellular matrix protein encoded by the FBN1 gene, is crucial for maintaining 
... | Miyadai R, Hinata S, Amemiya Y, Shigematsu S, Umeyama K, Nagashima H, Yamatoya K, Ohgane J |
| Endothelial Cell Senescence in Marfan Syndrome: Pathogenesis and Therapeutic Potential of TGF-β Pathway 
Inhibition. | Marfan syndrome (MFS) is a heritable connective tissue disorder caused by mutations ... | Chen Y, Zhu
Y, Ren X, Ding L, Xu Y, Zhou M, Dong R, Jin P, Chen X, Fan X, Li M, Gong Y, Wang Y |

[Step 4: Duration 11.29 seconds| Input tokens: 24,540 | Output tokens: 915]

| Title | Abstract | Authors |
|-------|----------|---------|
| Proximal stenting to resolve malperfusion before arch repair of a Marfan Type A dissection. | Malperfusion has been reported as one of the strongest predictors of morbidity ... | Potti C, Henn M, Orion K, Neschis M, Bozinovski J, Tillman BW |
| Epigenome editing-mediated restoration of FBN1 expression by demethylation of CpG island shore in porcine fibroblasts. | Fibrillin-1, an extracellular matrix protein encoded by the FBN1 gene, is crucial for maintaining ... | Miyadai R, Hinata S, Amemiya Y, Shigematsu S, Umeyama K, Nagashima H, Yamatoya K, Ohgane J |
| Endothelial Cell Senescence in Marfan Syndrome: Pathogenesis and Therapeutic Potential of TGF-β Pathway Inhibition. | Marfan syndrome (MFS) is a heritable connective tissue disorder caused by mutations ... | Chen Y, Zhu Y, Ren X, Ding L, Xu Y, Zhou M, Dong R, Jin P, Chen X, Fan X, Li M, Gong Y, Wang Y |



### Using two MCP tools in combination: brave and yahoo finance

In [9]:
# Load tools from MCP servers
with ToolCollection.from_mcp(yfinance_mcp_server, trust_remote_code=True) as yfinance_tool_collection, ToolCollection.from_mcp(brave_search_mcp_server, trust_remote_code=True) as brave_tool_collection :
   all_tools = [*brave_tool_collection.tools, *yfinance_tool_collection.tools]
   # Create an agent with all tools 
   agent = CodeAgent(tools=all_tools, model=model, add_base_tools=True, additional_authorized_imports=["time", "numpy", "scipy.*"])
   # Visualize agent behavior
   agent.visualize()
   print(agent.run("Calculate the last 5 days trend slope for NVDA stock and evaluate the result, then search for the latest news in relation to your evaluation."))

CodeAgent | openai/gpt-4o
├── ✅ Authorized imports: ['time', 'numpy', 'scipy.*']
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name               ┃ Description                                ┃ Arguments                                 ┃
    ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ brave_web_search   │ Performs a web search using the Brave      │ query (`string`): Search query (max 400   │
    │                    │ Search API, ideal for general queries,     │ chars, 50 words)                          │
    │                    │ news, articles, and online content. Use    │ count (`number`): Number of results       │
    │                    │ this for broad information gathering,      │ (1-20, default 10)                        │
    │                    │ recent events, or when you need diverse    │ offset (`number`): Pagination offset (max │
    │                    │ web sources. Supports pagination, content  │ 9, default 0)                             │
    │                    │ filtering, and freshness controls. Maximum │                                           │
    │                    │ 20 results per request, with offset for    │                                           │
    │                    │ pagination.                                │                                           │
    │ brave_local_search │ Searches for local businesses and places   │ query (`string`): Local search query      │
    │                    │ using Brave's Local Search API. Best for   │ (e.g. 'pizza near Central Park')          │
    │                    │ queries related to physical locations,     │ count (`number`): Number of results       │
    │                    │ businesses, restaurants, services, etc.    │ (1-20, default 5)                         │
    │                    │ Returns detailed information including:    │                                           │
    │                    │ - Business names and addresses             │                                           │
    │                    │ - Ratings and review counts                │                                           │
    │                    │ - Phone numbers and opening hours          │                                           │
    │                    │ Use this when the query implies 'near me'  │                                           │
    │                    │ or mentions specific locations.            │                                           │
    │                    │ Automatically falls back to web search if  │                                           │
    │                    │ no local results are found.                │                                           │
    │ get_ticker_info    │ Retrieve stock data including company      │ symbol (`string`): The stock symbol       │
    │                    │ info, financials, trading metrics and      │                                           │
    │                    │ governance data.                           │                                           │
    │ get_ticker_news    │ Fetches recent news articles related to a  │ symbol (`string`): The stock symbol       │
    │                    │ specific stock symbol with title, content, │                                           │
    │                    │ and source details.                        │                                           │
    │ search             │ Fetches and organizes search results from  │ query (`string`): The search query        │
    │                    │ Yahoo Finance, including stock quotes and  │ (ticker symbol or company name)           │
    │                    │ news articles.                             │ search_type (`string`): Type of search    │
    │                    │                                            │ resul

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the last 5 days trend slope for NVDA stock and evaluate the result, then search for the latest news   │
│ in relation to your evaluation.                                                                                 │
│                                                                                                                 │
╰─ LiteLLMModel - openai/gpt-4o ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To solve this task, the first step is to fetch the historical price data for the NVDA stock over the 
past 5 days. After obtaining this data, I will calculate the trend slope. Once the trend slope is determined, I 
will evaluate whether the trend is increasing or decreasing and use that evaluation to guide the news search. 

I will start by fetching the price history for NVDA for the last 5 days. Let's proceed with that.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 2.28 seconds| Input tokens: 2,671 | Output tokens: 91]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The upward trend in NVDA stock may be influenced by discussions around easing of U.S. chip export  
  restrictions and positive market analysis suggesting a higher average future price. This aligns with the trend   
  slope of approximately 0.898, indicating an upward trend.")                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The upward trend in NVDA stock may be influenced by discussions around easing of U.S. chip 
export restrictions and positive market analysis suggesting a higher average future price. This aligns with the 
trend slope of approximately 0.898, indicating an upward trend.

[Step 5: Duration 4.23 seconds| Input tokens: 17,249 | Output tokens: 631]

The upward trend in NVDA stock may be influenced by discussions around easing of U.S. chip export restrictions and positive market analysis suggesting a higher average future price. This aligns with the trend slope of approximately 0.898, indicating an upward trend.


Using an `ollama` local model:

In [13]:
# Define ollama model
model = LiteLLMModel(
    model_id="ollama_chat/qwen3:30b",
    api_base="http://localhost:11434", 
    api_key="ollama",
    num_ctx=32000,
)

# Load tools from MCP servers
with ToolCollection.from_mcp(yfinance_mcp_server, trust_remote_code=True) as yfinance_tool_collection, ToolCollection.from_mcp(brave_search_mcp_server, trust_remote_code=True) as brave_tool_collection :
   all_tools = [*brave_tool_collection.tools, *yfinance_tool_collection.tools]
   # Create an agent with all tools 
   agent = CodeAgent(tools=all_tools, model=model, add_base_tools=True, additional_authorized_imports=["time", "numpy", "scipy.*"])
   # Visualize agent behavior
   agent.visualize()
   print(agent.run("Calculate the last 5 days trend slope for NVDA stock closing price and evaluate the result, then search for the latest news based on this evaluation."))

CodeAgent | ollama_chat/qwen3:30b
├── ✅ Authorized imports: ['time', 'numpy', 'scipy.*']
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name               ┃ Description                                ┃ Arguments                                 ┃
    ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ brave_web_search   │ Performs a web search using the Brave      │ query (`string`): Search query (max 400   │
    │                    │ Search API, ideal for general queries,     │ chars, 50 words)                          │
    │                    │ news, articles, and online content. Use    │ count (`number`): Number of results       │
    │                    │ this for broad information gathering,      │ (1-20, default 10)                        │
    │                    │ recent events, or when you need diverse    │ offset (`number`): Pagination offset (max │
    │                    │ web sources. Supports pagination, content  │ 9, default 0)                             │
    │                    │ filtering, and freshness controls. Maximum │                                           │
    │                    │ 20 results per request, with offset for    │                                           │
    │                    │ pagination.                                │                                           │
    │ brave_local_search │ Searches for local businesses and places   │ query (`string`): Local search query      │
    │                    │ using Brave's Local Search API. Best for   │ (e.g. 'pizza near Central Park')          │
    │                    │ queries related to physical locations,     │ count (`number`): Number of results       │
    │                    │ businesses, restaurants, services, etc.    │ (1-20, default 5)                         │
    │                    │ Returns detailed information including:    │                                           │
    │                    │ - Business names and addresses             │                                           │
    │                    │ - Ratings and review counts                │                                           │
    │                    │ - Phone numbers and opening hours          │                                           │
    │                    │ Use this when the query implies 'near me'  │                                           │
    │                    │ or mentions specific locations.            │                                           │
    │                    │ Automatically falls back to web search if  │                                           │
    │                    │ no local results are found.                │                                           │
    │ get_ticker_info    │ Retrieve stock data including company      │ symbol (`string`): The stock symbol       │
    │                    │ info, financials, trading metrics and      │                                           │
    │                    │ governance data.                           │                                           │
    │ get_ticker_news    │ Fetches recent news articles related to a  │ symbol (`string`): The stock symbol       │
    │                    │ specific stock symbol with title, content, │                                           │
    │                    │ and source details.                        │                                           │
    │ search             │ Fetches and organizes search results from  │ query (`string`): The search query        │
    │                    │ Yahoo Finance, including stock quotes and  │ (ticker symbol or company name)           │
    │                    │ news articles.                             │ search_type (`string`): Type of search    │
    │                    │                                           

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the last 5 days trend slope for NVDA stock closing price and evaluate the result, then search for the │
│ latest news based on this evaluation.                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen3:30b ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get historical price data for NVDA                                                                             
  price_history = get_price_history(symbol="NVDA", period="5d", interval="1d")                                     
  print("Price history:", price_history)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Price history: | Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   
Stock Splits |
|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|
| 2025-05-02 00:00:00-04:00 | 114.18 | 115.4  | 113.37 |  114.5  | 1.90195e+08 |           0 |              0 |
| 2025-05-05 00:00:00-04:00 | 112.91 | 114.67 | 112.66 |  113.82 | 1.33163e+08 |           0 |              0 |
| 2025-05-06 00:00:00-04:00 | 111.48 | 114.74 | 110.82 |  113.54 | 1.58526e+08 |           0 |              0 |
| 2025-05-07 00:00:00-04:00 | 113.05 | 117.68 | 112.28 |  117.06 | 2.07828e+08 |           0 |              0 |
| 2025-05-08 00:00:00-04:00 | 118.25 | 118.68 | 115.85 |  117.37 | 1.97913e+08 |           0 |              0 |

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The 5-day trend slope for NVDA is 0.90 (positive), indicating an upward trend. Recent news         
  highlights analyst optimism, AI leadership, and market demand as key drivers.")                                  
                                                                                                                   
  final_answer("The 5-day trend slope for NVDA is 0.90 (positive), indicating an upward trend. Recent news         
  highlights analyst optimism, AI leadership, and market demand as key drivers.")                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The 5-day trend slope for NVDA is 0.90 (positive), indicating an upward trend. Recent news 
highlights analyst optimism, AI leadership, and market demand as key drivers.

[Step 4: Duration 24.53 seconds| Input tokens: 20,008 | Output tokens: 2,579]

The 5-day trend slope for NVDA is 0.90 (positive), indicating an upward trend. Recent news highlights analyst optimism, AI leadership, and market demand as key drivers.
